In [1]:
import os
import shutil
import numpy as np
from PIL import Image

<h3>SVM</h3>
<p>SVM Classifier with Histograms of Oriented Gradients (HOGs) and Local Binary Patterns (LBP)</p>
<p>To load weights into model for CNN + SVM, scroll to CNN-SVM section below</p>

In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import hog, local_binary_pattern
from scipy.spatial.distance import euclidean
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC


def extract_hog_features(image, orientations=9):
    hog_features, hog_image = hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2), 
                                   block_norm='L2-Hys', visualize=True, feature_vector=True)
    return hog_features

def extract_lbp_features(image):
    # Set the parameters for LBP
    radius = 1
    n_points = 8 * radius
    lbp = local_binary_pattern(image, n_points, radius, method="uniform")
    
    # Compute the histogram of LBP
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    # Normalize the histogram
    hist = hist.astype("float")
    hist /= hist.sum()
    
    return hist

def crop_center(image, crop_size=128):
    h, w = image.shape
    center_h, center_w = h // 2, w // 2
    half_crop = crop_size // 2
    cropped_image = image[center_h - half_crop:center_h + half_crop, center_w - half_crop:center_w + half_crop]
    return cropped_image

X = []
y = []
parent_path = "archive/similar/inpainting"

for folder_name in os.listdir(parent_path):
    folder_path = os.path.join(parent_path, folder_name)
    if os.path.isdir(folder_path):
        real_img = None
        forged_img = None
        print("reading folder " + folder_name)
        for image in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image)
            if os.path.isfile(image_path):
                img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

                if img is not None:
                    h, w = img.shape

                    # Skip images with height or width smaller than 256px
                    if h < 128 or w < 128:
                        print(f"Skipping image {image_path} (size: {h}x{w})")
                        continue  # Skip this image

                    if image.startswith("original"):
                        real_img = img
                    elif image.startswith("inpainting"):
                        forged_img = img

        if real_img is not None and forged_img is not None:
            real_img_cropped = crop_center(real_img)
            forged_img_cropped = crop_center(forged_img)

            real_hog = extract_hog_features(real_img_cropped)
            forged_hog = extract_hog_features(forged_img_cropped)

            real_lbp = extract_lbp_features(real_img_cropped)
            forged_lbp = extract_lbp_features(forged_img_cropped)

            combined_features = np.hstack((real_hog, real_lbp))
            X.append(combined_features)  
            y.append(0)  

            combined_features = np.hstack((forged_hog, forged_lbp))
            X.append(combined_features)
            y.append(1)  

X = np.array(X)
y = np.array(y)

In [ ]:
import os
import cv2
import numpy as np
from skimage.feature import hog, local_binary_pattern
from scipy.spatial.distance import euclidean
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
model_map = {"svm_linear": SVC(kernel = "linear"),
             "svm_rbf": SVC(kernel = "rbf"),
            "svm_poly": SVC(kernel = "poly")}

parameters_map = { 
    "C": [0.01, 0.1, 1, 10, 100],
    "gamma": [0.001, 0.01, 0.1, 1],
    "degree": [2, 3, 4, 5] 
}

def fine_tune(model: str, param_grid: dict, folds:int = 3):
    return HalvingGridSearchCV(model_map[model], param_grid, n_jobs = -1, refit = True, cv = folds, verbose = 3, factor = 3, resource = "n_samples")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

<b>SVM Model with Linear Kernel</p>

In [ ]:
# linear
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split, HalvingGridSearchCV
linear_svm = fine_tune("svm_linear", parameters_map)
linear_svm.fit(X_train, y_train)

In [ ]:
print(f"Grid search linear SVM best parameters: {linear_svm.best_params_}")
print(f"Cross validated score of the best linear SVM model: {linear_svm.best_score_}")

In [ ]:
y_pred = linear_svm.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred)  
print(f"Accuracy: {accuracy * 100:.2f}%")

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

<b>SVM Model with Polynomial Kernel</b>

In [ ]:
# polynomial
poly_svm = fine_tune("svm_poly", parameters_map)
poly_svm.fit(X_train, y_train)

In [ ]:
print(f"Grid search linear SVM best parameters: {poly_svm.best_params_}")
print(f"Cross validated score of the best linear SVM model: {poly_svm.best_score_}")

In [ ]:
y_pred_poly = poly_svm.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred_poly)  
print(f"Accuracy: {accuracy * 100:.2f}%")

conf_matrix = confusion_matrix(y_test, y_pred_poly)
print("Confusion Matrix:")
print(conf_matrix)

precision = precision_score(y_test, y_pred_poly)
recall = recall_score(y_test, y_pred_poly)
f1 = f1_score(y_test, y_pred_poly)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

<b>SVM Model with RBF Kernel</b>

In [ ]:
# rbf
rbf_svm = fine_tune("svm_rbf", parameters_map)
rbf_svm.fit(X_train, y_train)

In [ ]:
print(f"Grid search linear SVM best parameters: {rbf_svm.best_params_}")
print(f"Cross validated score of the best linear SVM model: {rbf_svm.best_score_}")

In [ ]:
y_pred_rbf = rbf_svm.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred_rbf)  
print(f"Accuracy: {accuracy * 100:.2f}%")

conf_matrix = confusion_matrix(y_test, y_pred_rbf)
print("Confusion Matrix:")
print(conf_matrix)

precision = precision_score(y_test, y_pred_rbf)
recall = recall_score(y_test, y_pred_rbf)
f1 = f1_score(y_test, y_pred_rbf)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")